In [10]:
import pandas as pd
import math

## Get data from usgs.gov

In [3]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')

### Parameters

In [15]:
# Lindsborg location, lat and long, degrees
centerLat = 38.5735
centerLong = -97.6745

# radians
cLat = centerLat * math.pi / 180.0
cLong = centerLong * math.pi / 180.0

# radius of earth, km
rad = 6371.0

### Distances

This is from sites about great circle distances:
- http://edwilliams.org/avform147.htm
- https://en.wikipedia.org/wiki/Great-circle_distance

In [26]:
def gcDist(centerLat, centerLong, ptLat, ptLong):
    '''Function to return great circle distance on earth, given center lat and long, point lat and long, all in degrees '''
    # go to radians
    cLat = centerLat * math.pi / 180.0
    cLong = centerLong * math.pi / 180.0
    pLat = ptLat * math.pi / 180.0
    pLong = ptLong * math.pi / 180.0

    # radius of earth, km
    rad = 6371.0

    # great circle formula
    dLambda = cLong - pLong
    dPhi = cLat - pLat

    drad = 2.0 * math.asin(math.sqrt( math.sin(dPhi/2.0)**2 + (1.0 - math.sin(dPhi/2.0)**2 - (math.sin((cLat+pLat)/2.0)**2) ) * (math.sin(dLambda/2.0)**2)  ) )

    # distance
    return drad * rad